# Flask Demo

In [1]:
import json
import os
import subprocess
import webbrowser

import psutil
import requests

In [2]:
# remove database file if exists to start fresh
if os.path.isfile("app.db"):
    os.remove("app.db")

In [3]:
# readable output for JSON
def print_readable_json(x: dict, indent: int = 2):
    print(json.dumps(x, indent=indent))

In [4]:
# start and track a uvicorn subprocess
flask_proc = subprocess.Popen(["python", "./app/flask_main.py"])
print("flask host process started with pid: {}".format(flask_proc.pid))

flask host process started with pid: 9392


In [5]:
print("flask pid exists: {}".format(psutil.pid_exists(flask_proc.pid)))

flask pid exists: True


In [6]:
target = "http://localhost:5000"

In [7]:
# test hello world
resp = requests.get(target)
print_readable_json(resp.json())

{
  "Hello": "World"
}


In [8]:
# make an items request
resp = requests.get("{}/items/1".format(target), params={"q": "first item description"})
print_readable_json(resp.json())

{
  "item_id": 1,
  "q": "first item description"
}


In [9]:
# send an improper request through
resp = requests.get(
    "{}/items/first".format(target), params={"q": "first item description"}
)
resp.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'

In [10]:
# get from the sklean dataset
resp = requests.get("{}/sklearn_wine".format(target))
sklearn_dataset = resp.json()
print_readable_json(sklearn_dataset[:1])

[
  {
    "alcalinity_of_ash": 15.6,
    "alcohol": 14.23,
    "ash": 2.43,
    "color_intensity": 5.64,
    "flavanoids": 3.06,
    "hue": 1.04,
    "id": 0,
    "magnesium": 127.0,
    "malic_acid": 1.71,
    "nonflavanoid_phenols": 0.28,
    "od280_od315_of_diluted_wines": 3.92,
    "proanthocyanins": 2.29,
    "proline": 1065.0,
    "total_phenols": 2.8
  }
]


In [11]:
# post records to the database
resp = requests.post("{}/sqlite_wine".format(target), data=json.dumps(sklearn_dataset))
resp.text
#print_readable_json(resp.json())

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>405 Method Not Allowed</title>\n<h1>Method Not Allowed</h1>\n<p>The method is not allowed for the requested URL.</p>\n'

In [12]:
# read from the database
resp = requests.get("{}/sqlite_wine".format(target), params={"limit": "2"})
resp.status_code
#results = resp.json()
#print_readable_json(results)

500

In [13]:
# kill the uvicorn subprocess
proc_killed = flask_proc.kill()

In [14]:
print("flask pid exists: {}".format(psutil.pid_exists(flask_proc.pid)))

flask pid exists: True
